Importing required libraries

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
from skimage.feature import blob_doh
import shutil
from csv import writer, reader
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from numpy import asarray, save, load

Uploading all data

In [ ]:
SOURCES_NUMBER = 1  # Should be equal for all data used
kSize = 5  # Size of smoothing kernel
kSigma = 1.5  # Sigma for smoothing
d = int((kSize-1)/2)
src_folder = "data/skymaps/IRF/"+str(SOURCES_NUMBER)
matrices = []
blob_pos = dict()
cont = 0
filename_dict = dict()
with open("data/xml_files/"+str(SOURCES_NUMBER)+"/blobs.csv") as csv_file:
    csv_reader = reader(csv_file, delimiter=',')
    tmp_rows = [row for row in csv_reader]
for filename in os.listdir(src_folder):
    if "IRF" in filename:
        filename_dict[int(filename.split("_")[2][:-5])] = filename
for order_id in sorted(filename_dict):
    blob_pos[cont] = []
    filename = filename_dict[order_id]
    matrices.append(fits.open(src_folder+"/"+filename))
    wcs = WCS(header=(matrices[-1])[0].header)
    for i in range(SOURCES_NUMBER):
        sky = SkyCoord(float(tmp_rows[cont][2*i+1]), float(tmp_rows[cont][2*i+2]), unit='deg')
        y, x= wcs.world_to_array_index(sky)
        blob_pos[cont].append([int(x), int(y)])
    cont += 1

Look at images

In [ ]:
for i, mat in enumerate(matrices[:5]):
    plt.matshow(mat[0].data, cmap='gray')

Defining methods required to smooth the images

In [ ]:
def gaussianKernel(size, sigma):
    kernel = np.fromfunction(lambda x, y: (1/(2*np.pi*sigma**2)) * np.e ** ((-1*((x-(size-1)/2)**2+(y-(size-1)/2)**2))/(2*sigma**2)), (size, size))
    return kernel

In [ ]:
def gaussianBlur(img, kernel):
    gaussian = np.zeros((img.shape[0]-2*d, img.shape[1]-2*d))
    for y in range(d, img.shape[0]-d):
        for x in range(d, img.shape[1]-d):
            gaussian[y-d][x-d] = np.sum(np.multiply(img[y-d:y+d+1, x-d:x+d+1], kernel))
    return gaussian

Smoothing and saving in each image in list "matrices_smoothed".

In [ ]:
# load numpy array from npy file
matrices_smoothed = load("data/skymaps/IRF/"+str(SOURCES_NUMBER)+"/matrices_smoothed.npy")
dimx = (matrices[-1][0].data).shape[0]-2*d
dimy = (matrices[-1][0].data).shape[1]-2*d
del matrices

In [ ]:
kernel = gaussianKernel(kSize, kSigma)
matrices_smoothed = []
for i, mat in enumerate(matrices):
    matrices_smoothed.append(gaussianBlur(mat[0].data, kernel))
dimx = (matrices[-1][0].data).shape[0]-2*d
dimy = (matrices[-1][0].data).shape[1]-2*d
del matrices

Save to npy file

In [ ]:
save("data/skymaps/IRF/"+str(SOURCES_NUMBER)+"/matrices_smoothed.npy", asarray(matrices_smoothed))

Showing smoothed images.

In [ ]:
print("Shape of smoothed image: ", matrices_smoothed[0].shape)

In [ ]:
for i, mat in enumerate(matrices_smoothed[:5]):
    plt.matshow(mat, cmap='gray')

Plot a histogram to choose threshold.

In [ ]:
plt.hist(matrices_smoothed[0].ravel(), bins=250, range=(0.0, 1.0));  # calculating histogram

In [ ]:
thresh = 0.35

Binarizing images using previously defined threshold.

In [ ]:
matrices_bin = []
for i, mat in enumerate(matrices_smoothed):
    matrices_bin.append(np.zeros((dimx, dimy)))
    matrices_bin[i][mat > thresh] = 1
del matrices_smoothed

In [ ]:
for mat in matrices_bin[:5]:
    plt.matshow(mat, cmap='gray')

Defining method used to erode images: convolution, padding and binary erosion.

In [ ]:
def conv(img, mask, logic):
    # Validation check
    try:
        if img.shape != mask.shape:
            raise Exception('image size don\'t fit the kernel size')
        if not (logic == 'AND' or logic == 'OR'):
            raise Exception(
                'parameter logic dosen\'t fit. Use \'AND\' or \'OR\' instead.')
    except:
        raise Exception('invaild data type:', type(img), '&', type(mask))

    # Conv
    imask = np.multiply(img, mask)

    # Result
    numOnekernel = len(np.where(mask == 1)[0])
    numOne = len(np.where(imask == 1)[0])
    if logic == 'AND':
        if numOne == numOnekernel:
            return 1
        return 0
    if logic == 'OR':
        if numOne == 0:
            return 0
        return 1


In [ ]:
def padding(img, kernel_size, border_filled):
    # Validation check
    try:
        img = np.array(img)
        if len(img.shape) != 2:
            raise Exception('Input shape of image doesn\'t fit.')
        if not (border_filled == 'CONSTANT' or border_filled == 'NEAREST'):
            raise Exception(
                'border_filled parameter dosen\'t fit. Use \'CONSTANT\' or \'NEAREST\' instead.'
            )
    except:
        raise Exception(
            'Invaild input image:expected ndarray or array-like data but get',
            type(img))

    # ZeroPadding
    x, y = img.shape
    dx, dy = kernel_size[0] // 2, kernel_size[1] // 2
    imgPadded = np.zeros((x + 2 * dx, y + 2 * dy))
    fx, fy = imgPadded.shape[0] - 1, imgPadded.shape[1] - 1
    imgPadded[dx:fx + 1 - dx, dy:fy + 1 - dy] = img

    # Nearest if needed
    if border_filled == 'NEAREST':
        for i in range(dy):
            imgPadded[dx:-dx, i] = imgPadded[dx:-dx, dy]
            imgPadded[dx:-dx, fy - i] = imgPadded[dx:-dx, fy - dy]
        for i in range(dx):
            imgPadded[i, dy:-dy] = imgPadded[dx, dy:-dy]
            imgPadded[fx - i, dy:-dy] = imgPadded[fx - dx, dy:-dy]
        imgPadded[:dx, :dy] = imgPadded[dx, dy]
        imgPadded[:dx, fy - dy:] = imgPadded[dx, fy - dy]
        imgPadded[fx - dx:, fy - dy:] = imgPadded[fx - dx, fy - dy]
        imgPadded[fx - dx:, :dy] = imgPadded[fx - dx, dy]
    return imgPadded

In [ ]:
def binErosion(img, kernel, border_filled='CONSTANT', logic='AND'):
    # Validation check
    try:
        img, kernel = np.array(img), np.array(kernel)
        if len(img.shape) != 2 or len(kernel.shape) != 2:
            raise Exception('Input shape of image or kernel doesn\'t fit.')
        if not (border_filled == 'CONSTANT' or border_filled == 'NEAREST'):
            raise Exception(
                'border_filled parameter dosen\'t fit. Use \'CONSTANT\' or \'NEAREST\' instead.'
            )
    except:
        raise Exception('invaild image type:expect ndarray but', type(img),
                        'or', type(kernel))

    x, y = img.shape
    dx, dy = kernel.shape
    imgOutput = np.zeros(img.shape)

    # ZeroPadding
    imgPadded = padding(img, kernel.shape, border_filled=border_filled)
    # Erosion Main
    for i in range(x):
        for j in range(y):
            imgOutput[i, j] = conv(
                imgPadded[i:i + dx, j:j + dy], kernel, logic=logic)
    #Return
    return imgOutput

Load numpy array from npy file

In [ ]:
matrices_dilated = []
matrices_eroded = load("data/skymaps/IRF/"+str(SOURCES_NUMBER)+"/matrices_eroded.npy")
del matrices_bin

Eroding and saving each image in list "matrices_eroded".

Showing images.

In [ ]:
matrices_eroded = []
matrices_dilated = []
for mat in matrices_bin:
    matrices_eroded.append(binErosion(copy.deepcopy(mat), np.array([[1,1], [1, 1]])))
    #plt.matshow(matrices_eroded[-1], cmap='gray')
del matrices_bin

Save to npy file

In [ ]:
save("data/skymaps/IRF/"+str(SOURCES_NUMBER)+"/matrices_eroded.npy", asarray(matrices_eroded))

Test with opening, but results are worst for current parameters.

In [ ]:
#for mat in matrices_eroded:
    #matrices_dilated.append(binErosion(copy.deepcopy(mat), np.array([[1,1], [1, 1]]), logic='OR'))
#del matrices_eroded

Extracting blobs using skimage.feature.blob_doh and evaluating number of blobs found.
If the number of blobs is equal to the declared value of "SOURCES_NUMBER" than the skymap will be saved in another folder (to be used as dataset) and the position of the blobs will be saved in a csv file in the same folder as the skymaps.

In [ ]:
blobs = dict()
mean_error_pix = []
mean_error_angle = []
data_taken = 0
matrices_to_use = matrices_dilated if len(matrices_dilated) > 0 else matrices_eroded
for i, mat in enumerate(matrices_to_use):
    blobs[i] = []
    cont = 0
    blobs_doh = blob_doh(mat, max_sigma=20, threshold=.01)
    for blob in blobs_doh:
        y, x, r = blob
        if r > 1:
            cont += 1
            blobs[i].append((int(x), int(y)))
    if cont == SOURCES_NUMBER:
        data_taken += 1
    else:
        blobs[i] = []
print("Percentage of images with correct amount of blobs: ", data_taken/len(matrices_to_use)*100, "%")
print("Showing data:")
with open("data/dataset_irf/blobs.csv", "a+", newline='') as write_obj:
    csv_writer = writer(write_obj)
    for key in blobs.keys():
        blobs_key = blobs[key]
        if len(blobs_key) == SOURCES_NUMBER:
            to_be_saved = [key, SOURCES_NUMBER]
            for blob in blobs_key:
                to_be_saved.append(blob)
                closer_blob_dist = 10000
                sky_obs = wcs.pixel_to_world(blob[0], blob[1])
                for i, blob_true in enumerate(blob_pos[key]):
                    sky_true = SkyCoord(float(tmp_rows[key][2*i+1]), float(tmp_rows[key][2*i+2]), unit='deg')
                    tmp_dist = (sky_true.separation(sky_obs)).degree
                    if tmp_dist < closer_blob_dist:
                        closer_blob_dist = tmp_dist
                        closer_blob = blob_true
                mean_error_angle.append(closer_blob_dist)
                mean_error_pix.append(np.sqrt((closer_blob[0]-blob[0])**2 + (closer_blob[1]-blob[1])**2))
                print("Error in pixel: ", mean_error_pix[-1])
                print("Error angle: ", mean_error_angle[-1])
            print("Coping skymap and blob ", blobs_key, " to 'dataset_irf' folder.")
            origin = src_folder+"/skymap_IRF_"+str(key)+".fits"
            dest = "data/dataset_irf/skymap_IRF_"+str(key)+"_"+str(SOURCES_NUMBER)+".fits"
            #shutil.copyfile(origin, dest)
            # Add contents of list as last row in the csv file
            #csv_writer.writerow(to_be_saved)
print("Mean error pixel: ", np.mean(mean_error_pix))
print("Max error in pixel: ", np.max(mean_error_pix))
print("Min error in pixel: ", np.min(mean_error_pix))
print("Standard deviation error pixel: ", np.std(mean_error_pix))
print("Mean error angle: ", np.mean(mean_error_angle))
print("Max error in angle: ", np.max(mean_error_angle))
print("Min error in angle: ", np.min(mean_error_angle))
print("Standard deviation error angle: ", np.std(mean_error_angle))